OPRE - Operations Research - Ostschweizer Fachhochschule (OST)

# Demonstration Knapsack Problem

Das Knapsack Problem ist das Beispiel, welches im Input Teil des Lernblocks 'Kombinatorische Optimierung' exemplarisch behandelt wird. Es kann daher verständlicherweise nicht mehr als Aufgabe für das Praxisprojekt ausgewählt werden. Im vorliegenden Jupyter Notebook werden die Inputs der Vorlesung mit Python-Code nachvollziehbar gemacht, damit Sie die vorgestellten Lösungsansätze (Konstruktionsheuristik, Verbesserungsheuristik, Randomisierte Lokale Suche, Mixed Integer Linear Program & Benchmarking) in der Praxis beobachten und nutzen zu lernen. Nutzen Sie dieses Jupyter Notebook als Begleiter und Referenz im Input Teil des Lernblocks und bei Bedarf während der Bearbeitung Ihres Praxisprojektes, um die Kern-Aspekte der einzelen Lösungsansätze nochmals in Erinnerung zu rufen.

**Inhaltsverzeichnis**
- [Knapsack Problemstellung](#problemstellung)
- [Daten Handling](#daten-handling)
    - [Datenformat](#datenformat)
    - [Readerfunktion](#reader-funktion)
    - [Testen der Reader-Funktion](#testen-der-reader-funktion)
    - [Problem Generator](#problem-generator)
- [Konstruktionsheuristik](#konstruktionsheuristik)
- [Verbesserungsheuristik](#verbesserungsheuristik-lokale-suche)
- [Randomisierte Lokale Suche](#randomisierte-lokale-suche-randomized-local-search)
- [Mixed Integer Linear Program & Benchmarking](#mixed-integer-linear-program--benchmarking)


## Problemstellung

Aus einer Menge von Objekten, die jeweils ein Gewicht und einen Nutzwert haben, soll eine Teilmenge ausgewählt werden, deren Gesamtgewicht eine Maximalkapazität nicht überschreitet. Unter dieser Bedingung soll der Nutzwert der ausgewählten Objekte maximiert werden.

<div style="text-align: center;">
  <img src="image.png" width="300">
</div>

## Daten Handling

### Datenformat
Um Problemstellungen generisch mit Hilfe von Dateien zu charakterisieren bieten sich Textdateien an, da sie menschenlesbar gestaltet werden können. In den Vorlesungsunterlagen finden Sie hinweise, wie sie dazu vorgehen können. Zur Beschreibung von Knapsack-Probleminstanzen müssen wir für jedes Item das Gewicht und den Wert kennen. Zusätzlich müssen wir die Rucksack-Kapazität angeben und der Instanz evtl. noch einen Namen und Kommentar zuweisen können. Ein mögliches Format dafür könnte folgendes sein: 

```
NAME: CAP_KS_3
TYPE: CAP_KNAPSACK
COMMENT: Pack 3 items into a knapsack with max_cap=15 (Leuthold)
NUM_ITEMS: 3
KNAPSACK_CAPACITY: 15
ITEMS_ID_WEIGHT_VALUE
1 9.0 10.0
2 7.0 6.0
3 7.0 6.0
EOF
```

Ist dieses definiert, können Sie mit Hilfe von ChatGPT einfach eine Reader-Funktion erzeugen, oder gar einen Problem-Generator schreiben lassen, der Ihnen beliebig viele und grosse Probleminstanzen generiert.

### Reader Funktion
Eine Reader Funktion wird dazu entwickelt, Probleminstanzen eines bestimmten Typs so einzulesen, damit wir die Informationen bequem weiterverwenden können. Nachfolgend ein Skript, das demonstriert, wie die von uns entworfene reader-Funktion für das Knapsack-Problem arbeitet:

In [1]:
from knapsack_reader import read_knapsack_problem

file = './data/CAP_KS_3.ks'

knapsack, capacity = read_knapsack_problem(file)  
print(f"Knapsack content: {knapsack}")
print(f"Knapsack capacity: {capacity}")


Knapsack content: {1: (9.0, 10.0), 2: (7.0, 6.0), 3: (7.0, 6.0)}
Knapsack capacity: 15.0


### Testen der Reader-Funktion

Mit einem Unit-Test können wir testen, ob wirklich auch die erwarteten Werte in den zurückgelieferten Datentypen enthalten sind. In Tests prüft man gewisse Erwartungen die man an den Code hat. Je mehr und genauer man testet, desto sicherer ist man, dass der Code das tut, was man auch erwartet. 

 Normalerweise schreibt man dafür Unit-Tests (z.B. für [pytest](https://docs.pytest.org/en/stable/)), hier als Demonstration jedoch ein Test explizit im Jupyter Notebook. Einen ähnlichen Unit-Test finden Sie in der Datei KNAPSACK_READER.py.

In [16]:
file_path = './data/CAP_KS_3.ks'
# read problem instance
items, capacity = read_knapsack_problem(file_path)
# make sure, datatype of items-variable is dict
assert type(items) == dict, "Expected items to be of type dict, but failed."
# make sure, the dict contains 3 entries
assert len(items) == 3, "Expected to find 5 items in dict items, but found " + str(len(items))
# make sure the capacity contains the expected value
assert capacity == 15, "Expected capacity to be 15 but got " + str(capacity)
# make sure the value of object 1 is 10 and weight is 9
assert items[1][1] == 10.0, "Expected value of item 1 to be 10 but got " + str(items[1][1])
# make sure the weight of object 1 is 9
assert items[1][0] == 9.0, "ArithmeticErrorExpected to be 9 but got " + str(items[1][0])

Der Test läuft duch, d.h. wir sind sicher, dass die getesteten Inhalte korrekt sind.

### Problem Generator
Nachfolgend eine kurze Präsentation eines Problemgenerators, der mit Hilfe von ChatGPT in wenigen Minuten entwickelt werden kann. Geben Sie das Datenformat für eine Testinstanz vor, kann er schnell einen Generator liefern, den Sie dann mit wenig Aufwand in die gewünschte Form bringen können, damit sinnvolle Wertebereiche abgedeckt werden.

In [17]:
from knapsack_prob_gen import create_knapsack_problem

knapsack = create_knapsack_problem(knapsack_capacity=15, num_items=3, value_range=(0.29, 38.73), test_name="CAP_KS")
print(knapsack)

NAME: CAP_KS_3
TYPE: CAP_KNAPSACK
COMMENT: Pack 3 items into a knapsack with max_cap=15 (Leuthold)
NUM_ITEMS: 3
KNAPSACK_CAPACITY: 15
ITEMS_ID_WEIGHT_VALUE
1 1.96 2.86
2 5.94 6.84
3 7.47 8.37
EOF



## Konstruktionsheuristik

Nachfolgend wird die im Unterricht entwickelte Konstruktionsheuristik demonstriert. Details dazu entnehmen Sie bitte dem zugehörigen Python-Skript und den Vorlesungsunterlagen. 

Das nachfolgende Skript liest eine Knapsack Probleminstanz ein und berechnet mit der Konstruktionsheuristik eine erste Startlösung. Danach wird diese ausgegeben:

In [18]:
from knapsack_kh import *

file = './data/CAP_KS_8.ks'

# Read the problem instance
items, knapsack_capacity = read_knapsack_problem(file)

# Create a first solution applying a primitive k-heuristic
solution, weight, value = knapsack_greedy(items, knapsack_capacity)

print(f"*** File {file}, with start node {id}:")
print(f"Num Items: {len(items)}")
print(f"Capacity: {knapsack_capacity}")
print("Solution:", [i + 1 for i, value in enumerate(solution) if value == 1])
print(f"Total Weight: {round(weight, 2)}")
print(f"Total Value: {round(value, 2)}")

*** File ./data/CAP_KS_8.ks, with start node <built-in function id>:
Num Items: 8
Capacity: 20.0
Solution: [3, 4, 5, 8]
Total Weight: 18.54
Total Value: 23.34


## Verbesserungsheuristik (Lokale Suche)

Nachfolgend wird die im Unterricht entwickelte Verbesserungsheuristik demonstriert. Details dazu entnehmen Sie bitte dem zugehörigen Python-Skript und den Vorlesungsunterlagen. 

Das nachfolgende Skript liest eine Knapsack Probleminstanz ein, löst sie zuerst mit der Konstruktionsheuristik (knapsack_greedy) und startet dann ausgehend von dieser Lösung eine Lokale Suche. Anschliesseng wird die gefundene Lösung ausgegeben:

In [19]:
from knapsack_kh_ls import *

file = './data/CAP_KS_8.ks'

# Read the problem instance
items, knapsack_capacity = read_knapsack_problem(file)

# Create a first solution applying a primitive k-heuristic
print(f"*** Solving Knapsack Instance File {file} ***")
solution, weight, value = knapsack_greedy(items, knapsack_capacity)
index_list = [i + 1 for i in range(len(items)) if solution[i] == 1]
print(f"Num Items: {len(items)}")
print(f"Capacity: {knapsack_capacity}")
print(f"Solution: {', '.join(map(str, index_list))}")
print(f"Total Weight: {round(weight, 2)}")
print(f"Total Value: {round(value, 2)}")

# now improve initial solution applying a best-improvement local-search
print(f"*** Improving solution with local search [from KH] ***")
solution, weight, value = knapsack_local_search(items, knapsack_capacity, solution, weight, value)
index_list = [i + 1 for i in range(len(items)) if solution[i] == 1]
print(f"Solution: {', '.join(map(str, index_list))}")
print(f"Total Weight: {round(weight, 2)}")
print(f"Total Value: {round(value, 2)}")

*** Solving Knapsack Instance File ./data/CAP_KS_8.ks ***
Num Items: 8
Capacity: 20.0
Solution: 3, 4, 5, 8
Total Weight: 18.54
Total Value: 23.34
*** Improving solution with local search [from KH] ***
Solution: 4, 5, 6, 8
Total Weight: 18.9
Total Value: 23.7


Durch die Anwendung der Lokalen Suche konnte die Lösung leicht verbessert werden. Anstelle vom Total Value von 23.34, den die Konstruktionsheuristik erzielen konnte, wurde durch die Lokale Suche eine bessere Lösung gefunden mit einem Wert von 23.7. Dazu wurde das Objekt 3 aus dem Rucksack entfernt und durch das Objekt 6 ersetzt.

## Randomisierte Lokale Suche (Randomized Local Search)

Nachfolgend wird die im Unterricht entwickelte Randomized Local Search demonstriert. Details dazu entnehmen Sie bitte dem zugehörigen Python-Skript und den Vorlesungsunterlagen. 

Das nachfolgende Skript liest eine Knapsack Probleminstanz ein, löst sie mit randomisierter lokaler Suche und gibt die gefundene Lösung aus:

In [20]:
from knapsack_kh_rnd_ls import *

file = './data/CAP_KS_8.ks'

# Read the problem instance
items, knapsack_capacity = read_knapsack_problem(file)

# Solve Instance with Randomized Local Search
solution, weight, value = knapsack_randomized_local_search(items, knapsack_capacity)
index_list = [i + 1 for i in range(len(items)) if solution[i] == 1]
print(f"Solution: {', '.join(map(str, index_list))}")
print(f"Total Weight: {round(weight, 2)}")
print(f"Total Value: {round(value, 2)}")

Solution: 4, 5, 6, 8
Total Weight: 18.9
Total Value: 23.7


Hier konnte die gewählte Instanz durch Randomized Local Search nicht weiter verbessert werden. Das ist aber kein Problem. Es reicht als Rechtfertigung völlit aus, wenn sie für einzelne Instanzen die Lösung verbessern kann. Es könnte ja auch sein, dass die durch die Lokale Suche gefundene Lösung bereits das Optimum ist?

## Mixed Integer Linear Program & Benchmarking

Wie im Unterricht erwähnt, wird eine Problemstellung in der Praxis häufig auch als ILP (MILP) modelliert. Der Vorteil von diesem Vorgehen liegt darin, dass man so oft für Probleminstanzen in überschaubarer Grösse optimale Lösungen berechnen kann. Falls alle praxisrelevanten Instanzen mit ILP gelöst werden können, fällt die Rechtfertigung und Motivation zur Entwicklung von Heuristiken in der Praxis i.d.R. weg. Meist sind die Probleme der kombinatorischen Optimierung aber so schwierig, dass für praxisrelevante Probleminstanzen nicht innert nützlicher Frist optimale Lösungen gefunden werden können. In diesen Fällen sind Heuristiken der Weg, um Lösungen zu finden. Die MILP-Formulierungen liefern dann mit ihren optimalen Lösungen für "einfache" Probleminstanzen einen Benchmark, der uns hilft, die Qualität der Lösungen unserer Heuristiken  (Optimality Gap) und damit die Heuristiken selbst beurteilen zu können.

Das nachfolgende Skript liest eine Knapsack Probleminstanz ein, löst sie mit einem MILP optimal und gibt die Lösung aus.

In [21]:
from knapsack_milp import *

file = './data/CAP_KS_8.ks'

# Read the problem instance
items, knapsack_capacity = read_knapsack_problem(file)

# Solve the knapsack problem using MILP
print(f"*** File {file}, with start node {id}:")
solution, weight, value = solve_knapsack_milp(items, knapsack_capacity)
index_list = [i + 1 for i in range(len(items)) if solution[i] == 1]
print(f"Num Items: {len(items)}")
print(f"Capacity: {knapsack_capacity}")
print(f"Solution: {', '.join(map(str, index_list))}")
print(f"Total Weight: {round(weight, 2)}")
print(f"Total Value: {round(value, 2)}")

*** File ./data/CAP_KS_8.ks, with start node <built-in function id>:
Optimal solution found.
Num Items: 8
Capacity: 20.0
Solution: 4, 5, 6, 8
Total Weight: 18.9
Total Value: 23.7


Jetzt wissen wir Bescheid. Die optimale Lösung der Instanz 'CAP_KS_8.ks' ist wirklich 23.7.